In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from requests import get
import itertools
import pandas as pd
import re

In [2]:
driver = webdriver.Firefox(executable_path=r'/Users/lilianboisse/Documents/PYTHON/M2/geckodriver')

In [3]:
adresse = "https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2019-12-31&sort=boxoffice_gross_us,desc&start="
adresse = "https://www.imdb.com/search/title/?title_type=feature&release_date=2000-01-01,2019-12-31&sort=num_votes,desc&start="

In [4]:
Mes_variables = ['Titre',
                 'Budget',
                 'Semaine 1',
                 'Box Mondial',
                 'Box USA',
                 'Producteur',
                 'Duree',
                 'Pays',
                 'Langues',
                 'Annee',
                 'Certification',
                 'Metascore',
                 'Note IMDB',
                 'Vote IMDB',
                 'Genre',
                 'Realisateurs' ,
                 'Nombre critiques'
                ]

In [5]:
def recuperation_lien(nb_page):
    '''Fonction permettant de récuperer tous les lien des films'''
    Liste_lien =[]
    nombre_page = [x for x in range(1, nb_page*50, 50)]
    for nb in nombre_page:
        driver.get(adresse + str(nb))
        recherche1 = driver.find_element_by_id("content-2-wide")
        recherche2 = recherche1.find_element_by_class_name("article")
        recherche3 = recherche2.find_element_by_class_name("lister-list")
        Liste_film = recherche3.find_elements_by_class_name("lister-item-content")
        for element in Liste_film:
            lien_film = element.find_element_by_partial_link_text('')
            Liste_lien.append(lien_film.get_attribute("href"))
    return Liste_lien

In [6]:
Liste_lien = recuperation_lien(150)

In [7]:
# Chemin pour les regex
patern_certi = re.compile(r'Certificate\:\s+(.*)')
patern_real = re.compile(r'Director[a-z]?\:\s+(.*)')
patern_country = re.compile(r'Country\:\s+(.*)\s+')
patern_langue = re.compile(r'Language\:\s+(.*)\s+')
patern_genre = re.compile(r'Genres\:\s+(.*)')
patern_budget = re.compile(r'Budget\:.*\$(\d+)')
patern_opening = re.compile(r'Opening Weekend USA\:.*\$(\d+)')
patern_usa_gross = re.compile(r'Gross USA\:.*\$(\d+)')
patern_world_gross = re.compile(r'Cumulative Worldwide Gross\:.*\$(\d+)')
patern_prod_comp = re.compile(r'Production Co\:\s+(.+)\,')
patern_duree = re.compile(r'(\d+).*min')
patern_nb_critic = re.compile(r'(\d+)\s+critic')

data={}
df=pd.DataFrame(columns=Mes_variables)
for film in Liste_lien:
    driver.get(film)
    
    #Recupereation des blocks utiles pour l'extraction des informations
    detail = driver.find_element_by_id("titleDetails")
    text_det = detail.text.replace(",", "")
    overview = driver.find_element_by_class_name("title-overview")
    block_titre = overview.find_element_by_class_name("title_block")
    ligne_annee = block_titre.find_element_by_id("titleYear")
    sous_text = block_titre.find_element_by_class_name("subtext")
    test = sous_text.text
    block_note_vote = block_titre.find_element_by_class_name("imdbRating")
    block_sommaire = overview.find_element_by_class_name("plot_summary ")
    block_storyline = driver.find_element_by_id('titleStoryLine').text
    recherche_O = overview.text
    
    # Recuperation des différente info grâce au block
    data["Note IMDB"] = block_note_vote.find_element_by_tag_name("strong").text
    data["Vote IMDB"] = block_note_vote.find_element_by_tag_name("a").text
    data["Annee"] = ligne_annee.find_element_by_tag_name("a").text
    data["Titre"] = block_titre.find_element_by_tag_name("h1").text
    
    try:
        data["Certification"] = patern_certi.findall(block_storyline)[0].split("|")[0].strip()
    except:
        data["Certification"]= None
    
    try:
        data["Realisateurs"] = patern_real.findall(recherche_O)[0]
    except:
        data["Realisateurs"]= None
    
    try:
        data["Pays"] = patern_country.findall(text_det)[0].split("|")[0].strip()
    except:
        data["Pays"]=None
        
    try:
        data["Langues"] = patern_langue.findall(text_det)[0].split("|")[0].strip()
    except:
        data["Langues"]= None
    
    try:
        data["Metascore"] = overview.find_element_by_class_name("metacriticScore").text
    except:
        data["Metascore"]= None 
    
    try:
        data["Genre"] = patern_genre.findall(block_storyline)[0]
    except:
        data["Genre"]=None
        
    try:
        data["Budget"] = patern_budget.findall(text_det)[0]
    except:
        data["Budget"]=None
        
    try:
        data["Semaine 1"] = patern_opening.findall(text_det)[0]
    except:
        data["Semaine 1"]=None
        
    try:
        data["Box USA"] = patern_usa_gross.findall(text_det)[0]
    except:
        data["Box USA"] =None
     
    try:
        data["Box Mondial"] = patern_world_gross.findall(text_det)[0]
    except:
        data["Box Mondial"]=None
        
    try:
        data["Producteur"] = patern_prod_comp.findall(detail.text)[0].split(",")[0].strip()
    except:
        data["Producteur"]= None
        
    try:
        data["Duree"] = patern_duree.findall(text_det)[0]
    except:
        data["Duree"]=None
        
    try:
        data["Nombre critiques"] = patern_nb_critic.findall(recherche_O)[0]
    except:
        data["Nombre critiques"]=None
    df=df.append(data,ignore_index=True)

In [8]:
df.to_csv(path_or_buf="/Users/lilianboisse/Documents/PYTHON/M2/film_imdb_final.csv", sep=";", index=False)